In [45]:
import pandas as pd
import sys
import pyreadstat
import glob
import tqdm

In [71]:
files = glob.glob("raw/*.xlsx")
files = [f for f in files if "Bilingual" in f]
files = [f for f in files if "R01" in f]
files = [f for f in files if "CDI" in f]

files

['raw/R01ED CDI Bilingual 4 Years, Master.xlsx',
 'raw/R01ED CDI Bilingual 3.5 Years, Master.xlsx',
 'raw/R01ED CDI Bilingual 2.5 Years, Master.xlsx',
 'raw/R01ED CDI Bilingual 3 Years, Master.xlsx']

In [72]:
## for the instruction file, use one of the files

df_dict = pd.read_excel(files[0], sheet_name=None, header=1)
all_df = []
for name,df in df_dict.items():
    if "Not in Use" in name:
        continue
    df = df.dropna(axis=1, how="all").fillna("")
    division_line_index = df[df["Testing Code"]==''].first_valid_index()
    df = df[:division_line_index]
    df["type"] = name
    all_df.append(df)
    print(df)

all_df = pd.concat(all_df)
    

      Testing Code R01ED400 R01ED402 R01ED403 R01ED408 R01ED409 R01ED411  \
0          baa baa      0.0      1.0      1.0      0.0      1.0      0.0   
1        choo choo      1.0      1.0      1.0      1.0      1.0      0.0   
2   cockadoodledoo      0.0      0.0      1.0      1.0      1.0      0.0   
3              grr      1.0      1.0      1.0      0.0      1.0      1.0   
4             meow      1.0      1.0      1.0      1.0      1.0      0.0   
5              moo      1.0      1.0      1.0      1.0      1.0      1.0   
6             ouch      0.0      1.0      1.0      1.0      1.0      0.0   
7      quack quack      1.0      1.0      1.0      1.0      1.0      1.0   
8            uh oh      0.0      1.0      1.0      1.0      1.0      1.0   
9            vroom      0.0      1.0      1.0      0.0      1.0      1.0   
10       woof woof      1.0      1.0      1.0      1.0      1.0      0.0   
11         yum yum      0.0      1.0      1.0      1.0      1.0      1.0   

   R01ED414

[103 rows x 96 columns]
   Testing Code R01ED400 R01ED402 R01ED403 R01ED408 R01ED409 R01ED411  \
0       allgone      1.0      0.0      1.0      0.0      1.0      0.0   
1        asleep      1.0      0.0      1.0      1.0      1.0      0.0   
2         awake      1.0      1.0      1.0      0.0      1.0      0.0   
3           bad      1.0      1.0      1.0      1.0      1.0      0.0   
4        better      1.0      1.0      1.0      0.0      1.0      0.0   
..          ...      ...      ...      ...      ...      ...      ...   
58          wet      1.0      1.0      1.0      0.0      1.0      0.0   
59        white      1.0      1.0      1.0      1.0      1.0      1.0   
60        windy      1.0      1.0      1.0      0.0      1.0      0.0   
61       yellow      1.0      1.0      1.0      1.0      1.0      1.0   
62        yucky      1.0      1.0      1.0      0.0      1.0      0.0   

   R01ED414 R01ED416 R01ED419  ... R2130Mo135 R2130Mo136 R2130Mo137  \
0       0.0      0.0      1.

In [73]:
l1 = all_df["Testing Code"].to_list() 
l2 = all_df["type"].to_list()

import numpy as np 
instruction = pd.DataFrame(np.array([l1,l2]).transpose(),columns=["item","type"])

In [74]:
instruction

,item,type
0,baa baa,Sound Effects and Animal Sounds
1,choo choo,Sound Effects and Animal Sounds
2,cockadoodledoo,Sound Effects and Animal Sounds
3,grr,Sound Effects and Animal Sounds
4,meow,Sound Effects and Animal Sounds
...,...,...
565,shoeses,C. Word Endings Part 2
566,sockses,C. Word Endings Part 2
567,teeths,C. Word Endings Part 2
568,toeses,C. Word Endings Part 2


In [75]:
choices = ["produces/does not produce"]*len(instruction)
instruction["choices"] = choices
instruction["itemID"] = ["item_"+str(i) for i in list(range(1,len(instruction["item"])+1))]
instruction["complexity_category"] = ['']*len(instruction["item"])
instruction["gloss"] = instruction["item"]

In [76]:
from collections import Counter
items = list(instruction.item)
uniq = []
for i in items:
    if i not in uniq:
        uniq.append(i)
    else:
        uniq.append(i+"_2")
instruction.item = uniq

In [77]:
len(set(instruction.item))

570

In [78]:
instruction.to_csv("Spanish_English_R01.csv", index=False)

In [79]:
instruction

,item,type,choices,itemID,complexity_category,gloss
0,baa baa,Sound Effects and Animal Sounds,produces/does not produce,item_1,,baa baa
1,choo choo,Sound Effects and Animal Sounds,produces/does not produce,item_2,,choo choo
2,cockadoodledoo,Sound Effects and Animal Sounds,produces/does not produce,item_3,,cockadoodledoo
3,grr,Sound Effects and Animal Sounds,produces/does not produce,item_4,,grr
4,meow,Sound Effects and Animal Sounds,produces/does not produce,item_5,,meow
...,...,...,...,...,...,...
565,shoeses,C. Word Endings Part 2,produces/does not produce,item_566,,shoeses
566,sockses,C. Word Endings Part 2,produces/does not produce,item_567,,sockses
567,teeths,C. Word Endings Part 2,produces/does not produce,item_568,,teeths
568,toeses,C. Word Endings Part 2,produces/does not produce,item_569,,toeses


In [80]:
## now loop through all kids to make the data files
all_ages = []
columns = instruction.itemID.to_list()

for file in files:
    age = file.split(" ")[-3]
    df_dict = pd.read_excel(files[0], sheet_name=None, header=1)
    all_df = []
    for name,df in df_dict.items():
        if "Not in Use" in name:
            continue
        df = df.dropna(axis=1, how="all").fillna("")
        division_line_index = df[df["Testing Code"]==''].first_valid_index()
        df = df[:division_line_index]
        all_df.append(df)
    all_df = pd.concat(all_df)
    df = all_df.transpose()
    df.columns = columns
    df["age"]=age
    df = df.drop("Testing Code",axis=0)
    df = df.reset_index()
    all_ages.append(df)
    
all_ages = pd.concat(all_ages)
all_ages
    

,index,item_1,item_2,item_3,item_4,item_5,item_6,item_7,item_8,item_9,...,item_562,item_563,item_564,item_565,item_566,item_567,item_568,item_569,item_570,age
0,R01ED400,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,4
1,R01ED402,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0,0,0,0,0,0,1,0,1,4
2,R01ED403,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0,1,1,0,0,0,1,0,0,4
3,R01ED408,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,...,0,0,0,0,0,0,0,0,0,4
4,R01ED409,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0,0,0,0,0,0,0,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,R2130Mo139,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0,0,1,0,0,0,0,0,0,3
90,R2130Mo141,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0,0,0,0,0,0,1,0,0,3
91,R2130Mo143,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,3
92,R2130Mo144,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,3


In [81]:
first_column = all_ages.pop('age')
all_ages.insert(1, 'age(years)', first_column)

In [82]:
first_column = all_ages.pop('index')
all_ages.insert(0, 'child', first_column)

In [83]:
all_ages

,child,age(years),item_1,item_2,item_3,item_4,item_5,item_6,item_7,item_8,...,item_561,item_562,item_563,item_564,item_565,item_566,item_567,item_568,item_569,item_570
0,R01ED400,4,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
1,R01ED402,4,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,...,1,0,0,0,0,0,0,1,0,1
2,R01ED403,4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0,0,1,1,0,0,0,1,0,0
3,R01ED408,4,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0
4,R01ED409,4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,R2130Mo139,3,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,...,1,0,0,1,0,0,0,0,0,0
90,R2130Mo141,3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0,0,0,0,0,0,0,1,0,0
91,R2130Mo143,3,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
92,R2130Mo144,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [84]:
all_ages.to_csv("Spanish-English(HDPM, Bilingual)_R01.csv",index=False)

In [85]:
subject = pd.read_excel("Participant information Hoff & Core R01 datafile.xlsx")
subject

,Participant_ID,Gender,Ethnicity,Language_Status,Father_education_30,Father_education_lang_30,Mother_education_30,Mother_education_lang_30,Chronological_age_30,Input_percent_Eng_home_30,Chronological_age_36,Input_percent_Eng_home_36,Chronological_age_42,Input_percent_Eng_home_42,Chronological_age_48,Input_percent_Eng_home_48
0,R01ED001,1,4,0,4,1,4,1,30.67,100.0,37.20,100,42.17,100,47.80,100
1,R01ED002,0,4,0,4,1,4,1,30.07,100.0,35.90,100,42.03,100,48.73,100
2,R01ED003,1,4,0,4,1,5,1,30.20,100.0,36.27,100,42.20,100,48.10,100
3,R01ED004,0,4,0,4,1,5,1,30.33,100.0,-997.00,-997,-997.00,-997,-997.00,-997
4,R01ED005,0,4,0,4,1,4,1,30.30,100.0,36.27,96,42.00,100,48.40,98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,R2130Mo141,1,2,1,4,2,4,2,30.67,50.0,36.07,30,42.63,25,48.03,40
220,R2130Mo143,0,2,1,2,1,2,2,30.10,0.0,36.10,0,42.77,0,48.90,10
221,R2130Mo144,1,2,1,2,1,5,1,30.40,20.0,36.50,20,41.90,10,48.20,10
222,R2130Mo145,1,2,1,3,1,4,2,30.93,0.0,36.27,0,42.50,0,49.00,5


In [86]:
subject.to_csv("Spanish_English_HDPM_data2_R01.csv",index=False)